# 智谱 AI 图像生成测试 Notebook（CogView-3-Flash 非对话模型，不支持思考模式）

本 Notebook 用于测试智谱 AI 的 CogView-3-Flash 图像生成功能。

**注意**：图像生成使用独立的 API 端点 `/images/generations`，不是 `/chat/completions`。

## 1. 环境配置

导入必要的库并配置 API 信息。

In [ ]:
# 导入必要的库
import os
import json
import requests
import base64
from PIL import Image
from io import BytesIO
from IPython.display import display, HTML
import time

# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4"
IMAGE_ENDPOINT = f"{BASE_URL}/images/generations"

print("✅ 环境配置完成")
print(f"API Base URL: {BASE_URL}")
print(f"Image Endpoint: {IMAGE_ENDPOINT}")

## 2. 图像生成类封装

封装图像生成功能，提供简洁的调用接口。

In [ ]:
class ZhipuImageGenerator:
    """智谱 AI 图像生成器"""
    
    def __init__(self, api_key: str, base_url: str = "https://open.bigmodel.cn/api/paas/v4"):
        """
        初始化图像生成器
        
        Args:
            api_key: API 密钥
            base_url: API 基础 URL
        """
        self.api_key = api_key
        self.base_url = base_url
        self.endpoint = f"{base_url}/images/generations"
    
    def generate(
        self,
        prompt: str,
        size: str = "1024x1024",
        n: int = 1,
        quality: str = "standard",
        style: str = None,
        response_format: str = "url"
    ) -> dict:
        """
        生成图像
        
        Args:
            prompt: 图像描述提示词
            size: 图像尺寸 (1024x1024, 1024x768, 768x1024, 512x512)
            n: 生成数量 (1-4)
            quality: 质量 (standard, hd)
            style: 风格 (natural, vivid, anime, oil_painting)
            response_format: 返回格式 (url, b64_json)
        
        Returns:
            API 响应数据
        """
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "model": "cogview-3-flash",
            "prompt": prompt,
            "size": size,
            "n": n,
            "quality": quality,
            "response_format": response_format
        }
        
        if style:
            payload["style"] = style
        
        try:
            response = requests.post(
                self.endpoint,
                headers=headers,
                json=payload,
                timeout=60
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return {"error": str(e)}
    
    def display_image_from_url(self, url: str, width: int = 512):
        """
        从 URL 加载并显示图像
        
        Args:
            url: 图像 URL
            width: 显示宽度
        """
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            
            # 调整显示尺寸
            ratio = width / img.width
            height = int(img.height * ratio)
            img_resized = img.resize((width, height), Image.Resampling.LANCZOS)
            
            display(img_resized)
            return img
        except Exception as e:
            print(f"❌ 图像加载失败: {e}")
            return None
    
    def display_image_from_b64(self, b64_string: str, width: int = 512):
        """
        从 Base64 加载并显示图像
        
        Args:
            b64_string: Base64 编码的图像数据
            width: 显示宽度
        """
        try:
            img_data = base64.b64decode(b64_string)
            img = Image.open(BytesIO(img_data))
            
            ratio = width / img.width
            height = int(img.height * ratio)
            img_resized = img.resize((width, height), Image.Resampling.LANCZOS)
            
            display(img_resized)
            return img
        except Exception as e:
            print(f"❌ 图像解码失败: {e}")
            return None
    
    def generate_and_display(
        self,
        prompt: str,
        display_width: int = 512,
        **kwargs
    ) -> list:
        """
        生成并显示图像
        
        Args:
            prompt: 提示词
            display_width: 显示宽度
            **kwargs: 其他生成参数
        """
        print(f"🎨 生成图像...")
        print(f"   提示词: {prompt[:80]}...")
        
        start_time = time.time()
        result = self.generate(prompt, **kwargs)
        elapsed = time.time() - start_time
        
        if "error" in result:
            print(f"❌ 生成失败: {result['error']}")
            return []
        
        print(f"✅ 生成完成! 耗时: {elapsed:.2f}秒")
        
        images = result.get("data", [])
        for i, img_data in enumerate(images):
            print(f"\n🖼️  图像 {i+1}:")
            
            if "url" in img_data:
                print(f"   URL: {img_data['url'][:80]}...")
                self.display_image_from_url(img_data["url"], display_width)
            elif "b64_json" in img_data:
                print("   格式: Base64")
                self.display_image_from_b64(img_data["b64_json"], display_width)
        
        return images

# 创建生成器实例
generator = ZhipuImageGenerator(API_KEY, BASE_URL)
print("✅ 图像生成器初始化完成")

## 3. 基础生成测试（风景图）

测试基本的图像生成功能，生成一张风景图。

In [ ]:
# 基础风景图测试
landscape_prompt = (
    "A serene mountain lake at sunrise, mist rising from the calm water, "
    "pine trees on the shore, reflection of snow-capped peaks, "
    "golden hour lighting, photorealistic, highly detailed, 8K resolution"
)

images = generator.generate_and_display(
    prompt=landscape_prompt,
    size="1024x1024",
    quality="standard",
    display_width=512
)

## 4. 不同尺寸测试

测试所有支持的图像尺寸。

In [ ]:
# 测试不同尺寸
sizes = ["1024x1024", "1024x768", "768x1024", "512x512"]
size_prompt = "A minimalist coffee cup on a wooden table, soft natural lighting, clean background, product photography style"

print("📐 测试不同尺寸生成\n" + "="*50)

for size in sizes:
    print(f"\n📏 尺寸: {size}")
    print("-"*50)
    
    result = generator.generate(
        prompt=size_prompt,
        size=size,
        quality="standard"
    )
    
    if "data" in result and len(result["data"]) > 0:
        img_url = result["data"][0].get("url", "")
        if img_url:
            print(f"✅ 生成成功: {img_url[:60]}...")
            # 加载并显示尺寸信息
            img = generator.display_image_from_url(img_url, width=300)
            if img:
                print(f"   实际尺寸: {img.size}")
    else:
        print(f"❌ 生成失败: {result.get('error', '未知错误')}")
    
    time.sleep(1)  # 避免请求过快

## 5. 批量生成测试

测试一次生成多张图像（n > 1）。

In [ ]:
# 批量生成测试
batch_prompt = "A cozy reading nook with warm lighting, bookshelves, comfortable armchair, and a cup of tea"

print("🔄 批量生成测试 (n=4)\n" + "="*50)

result = generator.generate(
    prompt=batch_prompt,
    size="1024x1024",
    n=4,  # 生成 4 张
    quality="standard"
)

if "data" in result:
    print(f"✅ 成功生成 {len(result['data'])} 张图像\n")
    
    for i, img_data in enumerate(result["data"]):
        print(f"图像 {i+1}:")
        if "url" in img_data:
            generator.display_image_from_url(img_data["url"], width=400)
        print()
else:
    print(f"❌ 批量生成失败: {result.get('error', '未知错误')}")

## 6. 不同风格测试

测试不同艺术风格的图像生成。

In [ ]:
# 不同风格测试
base_prompt = "A mystical ancient temple hidden in a foggy forest with glowing magical crystals"

styles = [
    ("natural", "自然/写实"),
    ("vivid", "鲜艳/数字艺术"),
    ("anime", "动漫"),
    ("oil_painting", "油画"),
]

print("🎨 不同风格测试\n" + "="*50)

for style_code, style_name in styles:
    print(f"\n🖌️ 风格: {style_name} ({style_code})")
    print("-"*50)
    
    prompt = f"{base_prompt}, {style_code} style, masterpiece, best quality"
    
    images = generator.generate_and_display(
        prompt=prompt,
        size="1024x1024",
        style=style_code,
        quality="standard",
        display_width=400
    )
    
    time.sleep(1)

## 7. 中文提示词测试

测试使用中文提示词的效果。

In [ ]:
# 中文提示词测试
chinese_prompts = [
    "一幅中国传统山水画，水墨风格，高山流水，云雾缭绕，松树挺拔",
    "江南水乡古镇，小桥流水，白墙黑瓦，春天桃花盛开",
    "一座宏伟的中国古代宫殿，金色屋顶，红色柱子，蓝天白云",
]

print("🇨🇳 中文提示词测试\n" + "="*50)

for prompt in chinese_prompts:
    print(f"\n提示词: {prompt}")
    print("-"*50)
    
    images = generator.generate_and_display(
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        display_width=400
    )
    
    time.sleep(1)

## 8. 高质量模式测试

对比标准质量和高清质量的生成效果。

In [ ]:
# 高清质量测试
quality_prompt = (
    "A majestic white tiger in a snowy forest, "
    "intricate fur details, piercing blue eyes, "
    "soft winter lighting, photorealistic, highly detailed"
)

qualities = ["standard", "hd"]

print("🔍 质量模式对比测试\n" + "="*50)

for quality in qualities:
    print(f"\n📊 质量模式: {quality.upper()}")
    print("-"*50)
    
    start_time = time.time()
    
    images = generator.generate_and_display(
        prompt=quality_prompt,
        size="1024x1024",
        quality=quality,
        display_width=400
    )
    
    elapsed = time.time() - start_time
    print(f"⏱️ 生成耗时: {elapsed:.2f}秒")
    
    time.sleep(1)

## 9. Base64 格式测试

测试 Base64 格式的图像返回。

In [ ]:
# Base64 格式测试
print("📦 Base64 格式测试\n" + "="*50)

b64_prompt = "A cute cartoon cat sleeping on a cushion, pastel colors, kawaii style"

result = generator.generate(
    prompt=b64_prompt,
    size="512x512",
    quality="standard",
    response_format="b64_json"
)

if "data" in result and len(result["data"]) > 0:
    b64_data = result["data"][0].get("b64_json", "")
    if b64_data:
        print("✅ Base64 数据接收成功")
        print(f"   数据长度: {len(b64_data)} 字符")
        print(f"   预览: {b64_data[:80]}...")
        
        # 解码并显示
        print("\n🖼️ 解码并显示图像:")
        generator.display_image_from_b64(b64_data, width=400)
else:
    print(f"❌ Base64 测试失败: {result.get('error', '未知错误')}")

## 10. 错误处理测试

测试各种错误情况的处理。

In [ ]:
# 错误处理测试
print("⚠️ 错误处理测试\n" + "="*50)

test_cases = [
    {
        "name": "空提示词",
        "params": {"prompt": "", "size": "1024x1024"}
    },
    {
        "name": "无效尺寸",
        "params": {"prompt": "a cat", "size": "9999x9999"}
    },
    {
        "name": "n 超出范围",
        "params": {"prompt": "a cat", "size": "1024x1024", "n": 10}
    },
]

for test_case in test_cases:
    print(f"\n🧪 测试: {test_case['name']}")
    print("-"*50)
    
    result = generator.generate(**test_case["params"])
    
    if "error" in result:
        print(f"✅ 错误被正确捕获")
        print(f"   错误信息: {result['error']}")
    elif "data" in result:
        print(f"⚠️ 请求成功 (可能 API 未拒绝此参数)")
        print(f"   返回: {result}")
    else:
        print(f"❓ 意外响应: {result}")

## 11. 综合场景测试

测试一些实际应用场景。

In [ ]:
# 综合场景测试
scenarios = [
    {
        "name": "产品展示",
        "prompt": "A sleek modern wireless earbuds case on a marble surface, professional product photography, soft studio lighting, minimalist style, 8K detail",
        "size": "1024x1024"
    },
    {
        "name": "游戏场景",
        "prompt": "An epic fantasy battlefield at sunset, dragons flying overhead, armies clashing, magical spells lighting up the sky, concept art style",
        "size": "1024x768"
    },
    {
        "name": "手机壁纸",
        "prompt": "A dreamy aurora borealis over a snowy mountain peak, stars twinkling in the night sky, vibrant colors, vertical composition",
        "size": "768x1024"
    },
    {
        "name": "社交媒体头像",
        "prompt": "A cute stylized robot character portrait, friendly expression, gradient background, digital art style, clean lines",
        "size": "512x512"
    },
]

print("🎯 综合场景测试\n" + "="*50)

for scenario in scenarios:
    print(f"\n📱 场景: {scenario['name']}")
    print(f"   尺寸: {scenario['size']}")
    print("-"*50)
    
    images = generator.generate_and_display(
        prompt=scenario["prompt"],
        size=scenario["size"],
        quality="hd",
        display_width=400
    )
    
    time.sleep(1)

## 12. 测试总结

总结本次测试的结果。

In [ ]:
# 测试总结
print("📋 测试总结\n" + "="*50)
print("""
✅ 完成的测试项:
1. 环境配置 - API 连接正常
2. 图像生成类封装 - 功能完整
3. 基础生成测试 - 风景图生成正常
4. 不同尺寸测试 - 1024x1024, 1024x768, 768x1024, 512x512
5. 批量生成测试 - 支持 n=1-4
6. 不同风格测试 - natural, vivid, anime, oil_painting
7. 中文提示词测试 - 支持中文输入
8. 高质量模式测试 - standard vs hd
9. Base64 格式测试 - 支持 url 和 b64_json
10. 错误处理测试 - 异常捕获正常
11. 综合场景测试 - 产品、游戏、壁纸、头像

📌 API 信息:
- 端点: https://open.bigmodel.cn/api/paas/v4/images/generations
- 模型: cogview-3-flash
- 免费使用

⚠️ 注意事项:
- 图像生成使用独立端点，非 /chat/completions
- 英文提示词效果通常更好
- 生成的图片 URL 有有效期，请及时保存
- 合理设置超时时间（建议 30-60 秒）
""")